# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

import warnings
warnings.filterwarnings("ignore")

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,papao,-17.7500,-149.5500,25.82,73,2,3.17,PF,2024-10-16
1,1,petropavlovsk-kamchatsky,53.0452,158.6483,8.33,66,20,7.00,RU,2024-10-16
2,2,adamstown,-25.0660,-130.1015,21.27,88,98,6.71,PN,2024-10-16
3,3,tual,-5.6667,132.7500,27.71,76,68,5.18,ID,2024-10-16
4,4,cockburn town,21.4612,-71.1419,29.34,71,9,7.79,TC,2024-10-16
5,5,lebu,-37.6167,-73.6500,9.92,84,99,2.15,CL,2024-10-16
6,6,la mira,18.0350,-102.3266,25.08,92,20,1.65,MX,2024-10-16
7,7,jamestown,42.0970,-79.2353,6.12,88,100,4.12,US,2024-10-16
8,8,bharatpur,27.2167,77.4833,29.91,25,17,2.29,IN,2024-10-16
9,9,puerto baquerizo moreno,-0.9000,-89.6000,20.53,89,59,5.60,EC,2024-10-16


In [4]:
df = pd.DataFrame(city_data_df)
df_count = df.groupby('City').size().reset_index(name='Count')
df_count


,City,Count
0,aasiaat,1
1,abbeville,1
2,acapulco de juarez,1
3,acarau,1
4,acton,1
...,...,...
572,yigo village,1
573,yorkton,1
574,yuzhno-kurilsk,1
575,ziro,1


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
%%capture --no-display

# Configure the map plot

# YOUR CODE HERE
Map_plot = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True, 
    tiles = 'OSM', 
    frame_width = 700, 
    frame_height = 500, 
    color='City',
    alpha= 0.5,
    s='Humidity',
    scale= 0.8)
# Display the map
# YOUR CODE 
Map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [8]:
df_max_temp = pd.DataFrame(city_data_df)
df_max_temp = df_max_temp[['City','Max Temp', 'Humidity', 'Wind Speed', 'Cloudiness']].head()
df_max_temp

,City,Max Temp,Humidity,Wind Speed,Cloudiness
0,papao,25.82,73,3.17,2
1,petropavlovsk-kamchatsky,8.33,66,7.00,20
2,adamstown,21.27,88,6.71,98
3,tual,27.71,76,5.18,68
4,cockburn town,29.34,71,7.79,9


In [9]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
ideal_weather_df = city_data_df.loc[
    (city_data_df['Max Temp'] >= 20) & (city_data_df['Max Temp'] <= 30) &  # Temperature between 20°C and 30°C
    (city_data_df['Humidity'] < 60) &     # Humidity less than 50%
    (city_data_df['Wind Speed'] < 10) &
    (city_data_df['Cloudiness'] < 50)
]
# Drop any rows with null values
# YOUR CODE HERE
ideal_weather_df = ideal_weather_df.dropna(how="any")

# Display sample data
# YOUR CODE HERE
ideal_weather_df.head(5)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,bharatpur,27.2167,77.4833,29.91,25,17,2.29,IN,2024-10-16
161,161,ad dawadimi,24.5077,44.3924,22.64,25,0,5.50,SA,2024-10-16
175,175,tchintabaraden,15.8969,5.7985,29.58,19,49,1.94,NE,2024-10-16
184,184,sabha,27.0377,14.4283,21.32,46,0,4.03,LY,2024-10-16
194,194,kidal,18.4411,1.4078,28.92,17,9,2.27,ML,2024-10-16


In [10]:
# Convert Celsius to Fahrenheit 
df_weather = pd.DataFrame(df_max_temp)
df_weather['Max Temp']= df_weather['Max Temp'] * 9/5 + 32
df_weather = df_weather[['City', 'Max Temp', 'Humidity', 'Wind Speed', 'Cloudiness']].head()
df_weather.head(50)


,City,Max Temp,Humidity,Wind Speed,Cloudiness
0,papao,78.476,73,3.17,2
1,petropavlovsk-kamchatsky,46.994,66,7.00,20
2,adamstown,70.286,88,6.71,98
3,tual,81.878,76,5.18,68
4,cockburn town,84.812,71,7.79,9


In [11]:
median_values = df_weather[['Max Temp', 'Humidity', 'Wind Speed', 'Cloudiness']].median()
median_values

Max Temp      78.476
Humidity      73.000
Wind Speed     6.710
Cloudiness    20.000
dtype: float64

In [12]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
ideal_df = city_data_df.loc[(city_data_df['Max Temp']<=30)\
                            &(city_data_df['Max Temp']>=20)\
                            &(city_data_df['Humidity']<70)
                            &(city_data_df['Cloudiness'] < 50)  
                            &(city_data_df['Wind Speed'] < 10),:]
print(len(ideal_df))

# Drop any rows with null values
ideal_df.dropna()

# Display sample data
ideal_df.head(5)

29


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,bharatpur,27.2167,77.4833,29.91,25,17,2.29,IN,2024-10-16
11,11,tura,25.5198,90.2201,27.58,61,0,1.15,IN,2024-10-16
155,155,flying fish cove,-10.4217,105.6791,27.82,69,40,6.69,CX,2024-10-16
161,161,ad dawadimi,24.5077,44.3924,22.64,25,0,5.50,SA,2024-10-16
175,175,tchintabaraden,15.8969,5.7985,29.58,19,49,1.94,NE,2024-10-16


In [13]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
ideal_df = city_data_df.loc[(city_data_df['Max Temp']<=30)\
                            &(city_data_df['Max Temp']>=20)\
                            &(city_data_df['Humidity']<70)
                            &(city_data_df['Cloudiness'] < 50)  
                            &(city_data_df['Wind Speed'] < 10),:]
print(len(ideal_df))

# Drop any rows with null values
ideal_df.dropna()

# Display sample data
ideal_df.head(5)

29


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,bharatpur,27.2167,77.4833,29.91,25,17,2.29,IN,2024-10-16
11,11,tura,25.5198,90.2201,27.58,61,0,1.15,IN,2024-10-16
155,155,flying fish cove,-10.4217,105.6791,27.82,69,40,6.69,CX,2024-10-16
161,161,ad dawadimi,24.5077,44.3924,22.64,25,0,5.50,SA,2024-10-16
175,175,tchintabaraden,15.8969,5.7985,29.58,19,49,1.94,NE,2024-10-16


### Step 3: Create a new DataFrame called `hotel_df`.

In [15]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = city_data_df.iloc[:,[1,8,2,3,4]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df['Hotel Name']=''

# Display sample data
# YOUR CODE HERE
hotel_df.head(10)


,City,Country,Lat,Lng,Max Temp,Hotel Name
0,papao,PF,-17.7500,-149.5500,25.82,
1,petropavlovsk-kamchatsky,RU,53.0452,158.6483,8.33,
2,adamstown,PN,-25.0660,-130.1015,21.27,
3,tual,ID,-5.6667,132.7500,27.71,
4,cockburn town,TC,21.4612,-71.1419,29.34,
5,lebu,CL,-37.6167,-73.6500,9.92,
6,la mira,MX,18.0350,-102.3266,25.08,
7,jamestown,US,42.0970,-79.2353,6.12,
8,bharatpur,IN,27.2167,77.4833,29.91,
9,puerto baquerizo moreno,EC,-0.9000,-89.6000,20.53,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [17]:
# Set parameters to search for a hotel
radius = 1000 # YOUR CODE HERE
params = {'categories':'accommodation.hotel',\
         'filter':'',\
         'bias':'',\
         'apiKey':geoapify_key,\
         'limit':1
    # YOUR CODE HERE
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    lon = row['Lng']
    lat = row['Lat']
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'# YOUR CODE HERE
    params["bias"] = f'proximity:{lon},{lat}'# YOUR CODE HERE

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params) # YOUR CODE HERE

    # Convert the API response to JSON format
    name_address = name_address.json() # YOUR CODE HERE

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
papao - nearest hotel: No hotel found
petropavlovsk-kamchatsky - nearest hotel: ООО Постоялый двор
adamstown - nearest hotel: No hotel found
tual - nearest hotel: No hotel found
cockburn town - nearest hotel: The Salt Raker Inn
lebu - nearest hotel: No hotel found
la mira - nearest hotel: No hotel found
jamestown - nearest hotel: DoubleTree Jamestown
bharatpur - nearest hotel: Hotel Jai Palace
puerto baquerizo moreno - nearest hotel: Dory's House
albany - nearest hotel: No hotel found
tura - nearest hotel: No hotel found
shimoda - nearest hotel: ホテルウラガ
grytviken - nearest hotel: No hotel found
qaqortoq - nearest hotel: Hotel Qaqortoq
touros - nearest hotel: Pousada Atlântico
port-aux-francais - nearest hotel: Keravel
edinburgh of the seven seas - nearest hotel: No hotel found
pangoa - nearest hotel: No hotel found
oranjemund - nearest hotel: No hotel found
loralai - nearest hotel: No hotel found
waitangi - nearest hotel: Hotel Chathams
margaret river - nearest hot

,City,Country,Lat,Lng,Max Temp,Hotel Name
0,papao,PF,-17.7500,-149.5500,25.82,No hotel found
1,petropavlovsk-kamchatsky,RU,53.0452,158.6483,8.33,ООО Постоялый двор
2,adamstown,PN,-25.0660,-130.1015,21.27,No hotel found
3,tual,ID,-5.6667,132.7500,27.71,No hotel found
4,cockburn town,TC,21.4612,-71.1419,29.34,The Salt Raker Inn


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [34]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot_2 = city_data_df.hvplot.points('Lng',
            'Lat',
            geo=True, 
            tiles = 'OSM', 
            frame_width = 700, 
            frame_height = 500, 
            color='City',
            alpha=0.6,
            hover=True,
            hover_cols='all',
            use_index=False,
            s='Humidity',
            scale= 1.2)

# Display the map
# YOUR CODE HERE
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,City_ID,Max Temp,Cloudiness,Wind Speed,Country,Date)